In [90]:
import emoji

In [91]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

In [92]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [93]:
f = open("glove.6B.50d.txt", encoding = 'utf-8')

In [94]:
import numpy as np

In [95]:
word_embedding = {}
for line in f:
    values = line.split()
    val = values[0]
    coff = np.asarray(values[1:],dtype = float)
    word_embedding[val] = coff
f.close()

In [96]:
emb_dim = word_embedding["the"].shape
print(emb_dim)

(50,)


In [97]:
import pandas as pd

In [98]:
train_data =pd.read_csv("train_emoji.csv")

In [99]:
x_train,y_train = train_data.iloc[:,0],train_data.iloc[:,1]

In [100]:
xtrain,ytrain = x_train.values,y_train.values

In [101]:
test_data = pd.read_csv("test_emoji.csv")
xtest = test_data.iloc[:,0].values

In [102]:
ytest = test_data.iloc[:,1].values

In [103]:
from keras.utils import np_utils
ytrain = np_utils.to_categorical(ytrain)

In [104]:
ytest = np_utils.to_categorical(ytest)

# Converting Sentences into Vectors

In [105]:
def sec2vec(x):
    maxlen = 10
    state_vec = np.zeros((x.shape[0],maxlen,emb_dim[0]))
    for i in range(x.shape[0]):
        x[i] = x[i].split()
        for j in range(len(x[i])):
            try:
                state_vec[i][j] = word_embedding[x[i][j].lower()]
            except:
                state_vec[i][j] = np.zeros((50,))
    return state_vec

In [106]:
embedding_mat_train = sec2vec(xtrain)
embedding_mat_test = sec2vec(xtest)

In [107]:
embedding_mat_test.shape

(55, 10, 50)

# Define LSTM model

In [156]:
import keras
from keras.models import Sequential
from keras.layers import *

In [162]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences = False))
model.add(Dense(5))
model.add(Activation("softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_16 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_12 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [163]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("best_model.h5",monitor = "val_loss",verbose = True,save_best_only=True)
earlystop = EarlyStopping(monitor = "val_acc",patience = 14)

In [164]:
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ["acc"])

In [165]:
model.fit(embedding_mat_train,ytrain,epochs = 100,batch_size = 64,shuffle = True,validation_split = 0.2,callbacks = [checkpoint,earlystop])

Train on 104 samples, validate on 27 samples
Epoch 1/100
104/104 [==============================] - 3s 32ms/step - loss: 1.6071 - acc: 0.2404 - val_loss: 1.6139 - val_acc: 0.1111

Epoch 00001: val_loss improved from inf to 1.61386, saving model to best_model.h5
Epoch 2/100
104/104 [==============================] - 0s 738us/step - loss: 1.5676 - acc: 0.3654 - val_loss: 1.6085 - val_acc: 0.1852

Epoch 00002: val_loss improved from 1.61386 to 1.60845, saving model to best_model.h5
Epoch 3/100
104/104 [==============================] - 0s 700us/step - loss: 1.5365 - acc: 0.3750 - val_loss: 1.6074 - val_acc: 0.2222

Epoch 00003: val_loss improved from 1.60845 to 1.60737, saving model to best_model.h5
Epoch 4/100
104/104 [==============================] - 0s 834us/step - loss: 1.4997 - acc: 0.4135 - val_loss: 1.6171 - val_acc: 0.1852

Epoch 00004: val_loss did not improve from 1.60737
Epoch 5/100
104/104 [==============================] - 0s 662us/step - loss: 1.4692 - acc: 0.4519 - val_los

In [167]:
model.load_weights("best_model.h5")
model.evaluate(embedding_mat_test,ytest)

55/55 [==============================] - 0s 308us/step


[1.2810721614144065, 0.5090909069234675]

In [174]:
pred = model.predict_classes(embedding_mat_test)

In [175]:
for i in range(30):
    print(' '.join(xtest[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(ytest[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
😁
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😁
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😓
Why are you feeling bad
😓
😓
I am upset
😓
⚾
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
❤️
valentine day is near
❤️
😁
I miss you so much
❤️
❤️
